In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import pytorch_lightning as pl
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
import torch

# 0. Prepare data
# assume df_cycle_2 loaded with datetime_ index
df = df_cycle_2.copy()
df.index = pd.to_datetime(df.index)
df = df.sort_index()

# reset index and create time_idx within each cycle (sequence position)
df = df.reset_index().rename(columns={"index": "datetime_"})
df["time_idx"] = df.groupby("cycle").cumcount() + 1  # starts at 1 per cycle

# features and targets
exog = ["Voltage_measured", "Current_measured", "Temperature_measured"]
targets = ["Time", "Capacity"]

# store forecasts for each cycle and each model
all_preds = {t: {"sarimax": [], "prophet": [], "lstm": [], "tft": []} for t in targets}

# list of cycles in order
cycles = sorted(df["cycle"].unique())

for cyc in cycles:
    # history = all previous cycles
    hist = df[df["cycle"] < cyc]
    future = df[df["cycle"] == cyc]

    if hist.empty:
        # cannot forecast the first cycle; append NaNs or copy actuals
        for y in targets:
            all_preds[y]["sarimax"].append(np.full(len(future), np.nan))
            all_preds[y]["prophet"].append(np.full(len(future), np.nan))
            all_preds[y]["lstm"].append(np.full(len(future), np.nan))
            all_preds[y]["tft"].append(np.full(len(future), np.nan))
        continue

    # 1. SARIMAX per target
    for y in targets:
        sar_model = SARIMAX(
            hist[y],
            exog=hist[exog],
            order=(1,1,1),
            enforce_stationarity=False,
            enforce_invertibility=False
        ).fit(disp=False)
        pred = sar_model.predict(
            start=future.index[0],
            end=future.index[-1],
            exog=future[exog]
        )
        all_preds[y]["sarimax"].append(pred.values)

    # 2. Prophet per target
    df_hist = hist[["datetime_", *targets, *exog]]
    for y in targets:
        df_p = df_hist[["datetime_", y] + exog].rename(columns={"datetime_": "ds", y: "y"})
        m = Prophet()
        for reg in exog:
            m.add_regressor(reg)
        m.fit(df_p)
        df_fut = future[["datetime_", *exog]].rename(columns={"datetime_": "ds"})
        fc = m.predict(df_fut)
        all_preds[y]["prophet"].append(fc["yhat"].values)

    # 3. LSTM per target (example: Capacity)
    arr = hist[["Capacity"] + exog].values
    scaler = MinMaxScaler().fit(arr)
    scaled = scaler.transform(arr)
    seq_len = min(20, len(scaled) - 1)
    def create_seq(a, l):
        X, y = [], []
        for i in range(l, len(a)):
            X.append(a[i-l:i])
            y.append(a[i, 0])
        return np.array(X), np.array(y)
    X_hist, y_hist = create_seq(scaled, seq_len)
    X_train, y_train = X_hist[:-seq_len], y_hist[:-seq_len]
    X_test = scaled[-seq_len:][None, ...]
    lstm = Sequential([LSTM(32, input_shape=(seq_len, X_hist.shape[2])), Dense(1)])
    lstm.compile('adam', 'mse')
    lstm.fit(X_train, y_train, epochs=5, verbose=0)
    cap_pred = scaler.inverse_transform(
        np.hstack([lstm.predict(X_test), np.zeros((1, len(exog)))])
    )[:, 0]
    all_preds["Capacity"]["lstm"].append(cap_pred)

    # 4. Temporal Fusion Transformer for Capacity
    hist_str = hist.copy()
    hist_str["cycle"] = hist_str["cycle"].astype(str)
    max_pred = future.shape[0]
    tft_ds = TimeSeriesDataSet(
        hist_str,
        time_idx="time_idx",
        target="Capacity",
        group_ids=["cycle"],
        time_varying_known_reals=["time_idx"] + exog,
        time_varying_unknown_reals=["Capacity"],
        target_normalizer=GroupNormalizer(groups=["cycle"]),
        max_encoder_length=30,
        max_prediction_length=max_pred
    )
    tft = TemporalFusionTransformer.from_dataset(
        tft_ds,
        hidden_size=16,
        attention_head_size=1,
        dropout=0.1,
        hidden_continuous_size=8,
        loss=torch.nn.MSELoss()
    )
    trainer = pl.Trainer(max_epochs=10, logger=False, enable_progress_bar=False)
    trainer.fit(tft, train_dataloaders=torch.utils.data.DataLoader(tft_ds, batch_size=64))
    df_fut = future.copy()
    df_fut["cycle"] = df_fut["cycle"].astype(str)
    raw = tft.predict(df_fut, mode='prediction', return_x=False)
    all_preds["Capacity"]["tft"].append(raw.numpy().flatten())

# Now all_preds dict contains per-cycle forecasts for each model and target
print("Finished forecasting all cycles for Time and Capacity.")
```
